<a href="https://colab.research.google.com/github/flake313/Malashin/blob/main/Another_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install pymupdf chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00


**Для проверки**

In [22]:
def detect_encoding_manually(byte_data):
    if byte_data.startswith(b'\xef\xbb\xbf'):
        return 'UTF-8 с BOM'
    if byte_data.startswith(b'\xff\xfe'):
        return 'UTF-16 LE'
    if byte_data.startswith(b'\xfe\xff'):
        return 'UTF-16 BE'

    # Проверка на UTF-8 без BOM
    try:
        byte_data.decode('utf-8')
        return 'UTF-8 без BOM'
    except UnicodeDecodeError:
        pass

    # Проверка на ASCII
    if all(byte <= 0x7F for byte in byte_data):
        return 'ASCII'

    # Проверка на Windows-1251
    try:
        byte_data.decode('windows-1251')
        return 'Windows-1251'
    except UnicodeDecodeError:
        pass

    # Проверка на Windows-1252
    try:
        byte_data.decode('windows-1252')
        return 'Windows-1252'
    except UnicodeDecodeError:
        pass

    return 'Неизвестная кодировка'

# Пример использования
byte_data = b'\xef\xbb\xbfHello, World!'  # UTF-8 с BOM
print(detect_encoding_manually(byte_data))  # Выведет: UTF-8 с BOM

byte_data = b'\xff\xfefile content'  # UTF-16 LE
print(detect_encoding_manually(byte_data))  # Выведет: UTF-16 LE

byte_data = b'Hello, World!'  # ASCII
print(detect_encoding_manually(byte_data))  # Выведет: ASCII

byte_data = b'\xd0\x9f\xd0\xb0\xd1\x80\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd1\x80'  # Windows-1251
print(detect_encoding_manually(byte_data))  # Выведет: Windows-1251

UTF-8 с BOM
UTF-16 LE
UTF-8 без BOM
UTF-8 без BOM


**Сама проверка**

In [25]:
import os

def test_encoding_detection():
    base_dir = ''
    test_files = {
        #os.path.join(base_dir, "WINDOWS1251_endPDF (1).pdf"): "UTF-8 с BOM",
        #os.path.join(base_dir, "UTF_RusInLaTeX_1.txt"): "UTF-8 без BOM",
        #os.path.join(base_dir, "_UTF-16LE_RusInLaTeX_1.txt"): "UTF-16 LE",
        #os.path.join(base_dir, "UTF_8 with BOM_RusInLaTeX_1.txt"): "UTF-16 BE",
        #os.path.join(base_dir, "converted.pdf"): "Windows-1251",
        #os.path.join(base_dir, "endPDF_Courier (1).pdf"): "Windows-1252",
        os.path.join(base_dir, "endPDF_Courier (1).pdf"): "Windows-1251",
        os.path.join(base_dir, "converted.pdf"): "UTF-8 без BOM",
    }

    for filename, expected in test_files.items():
        with open(filename, 'rb') as f:
            byte_data = f.read()
            detected = detect_encoding_manually(byte_data)
            print(f"Файл: {filename}, Ожидаемая кодировка: {expected}, Обнаруженная кодировка: {detected}")

if __name__ == "__main__":
    test_encoding_detection()

Файл: endPDF_Courier (1).pdf, Ожидаемая кодировка: Windows-1251, Обнаруженная кодировка: Windows-1251
Файл: converted.pdf, Ожидаемая кодировка: UTF-8 без BOM, Обнаруженная кодировка: UTF-8 без BOM


**Перекодировка**

In [17]:
import fitz  # PyMuPDF
import chardet
import io

def convert_text_from_windows1251_to_utf8(text: bytes) -> str:
    # Проверяем текущую кодировку текста
    detected_encoding = chardet.detect(text)['encoding']
    if detected_encoding == 'windows-1251':
        return text.decode('windows-1251').encode('utf-8').decode('utf-8')
    return text.decode('utf-8')

def convert_pdf(input_pdf_path: str, output_pdf_path: str):
    # Открываем исходный PDF файл
    pdf_document = fitz.open(input_pdf_path)

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text_instances = page.get_text("dict")["blocks"]

        for block in text_instances:
            if block["type"] == 0:  # 0 - текстовый блок
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Конвертируем текст из Windows-1251 в UTF-8
                        span["text"] = convert_text_from_windows1251_to_utf8(span["text"].encode('utf-8'))

    # Сохраняем измененный PDF файл
    pdf_document.save(output_pdf_path)
    pdf_document.close()

if __name__ == "__main__":
    input_pdf_path = "endPDF_Courier (1).pdf"  # Путь к исходному PDF файлу
    output_pdf_path = "converted.pdf"  # Путь к конвертированному PDF файлу

    convert_pdf(input_pdf_path, output_pdf_path)
    print(f"PDF файл успешно конвертирован и сохранен как {output_pdf_path}")

PDF файл успешно конвертирован и сохранен как converted.pdf
